In [1]:
import pandas as pd

df = pd.read_csv('../../output_data/4_2_vectorized_ds_glove_25.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25763 entries, 0 to 25762
Columns: 1102 entries, Unnamed: 0 to tag
dtypes: float64(1100), int64(1), object(1)
memory usage: 216.6+ MB


In [2]:
df.head()

,Unnamed: 0,input_0,input_1,input_2,input_3,input_4,input_5,input_6,input_7,input_8,...,input_1091,input_1092,input_1093,input_1094,input_1095,input_1096,input_1097,input_1098,input_1099,tag
0,0,0.035234,0.54248,-0.75981,-0.029171,1.94140,-0.17883,0.25397,0.49265,-0.00256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COMMUNITY
1,1,-0.398190,0.92849,1.11940,-0.132170,-0.29808,-0.52848,1.06140,-0.70803,-0.48539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENVIRONMENT
2,2,-0.942140,-0.24345,0.20744,0.214930,-1.06640,1.19490,1.52340,-0.16527,0.35522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RELAX
3,3,-1.074400,-0.88249,0.27640,0.647550,-0.35475,0.25591,-0.70326,-1.00450,1.19360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRAVEL
4,4,-0.177000,0.43200,0.21391,-0.295150,0.47344,-0.38013,1.60180,0.56238,0.40478,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASTRONOMY


In [3]:
df = df.fillna(0)

df.head()

,Unnamed: 0,input_0,input_1,input_2,input_3,input_4,input_5,input_6,input_7,input_8,...,input_1091,input_1092,input_1093,input_1094,input_1095,input_1096,input_1097,input_1098,input_1099,tag
0,0,0.035234,0.54248,-0.75981,-0.029171,1.94140,-0.17883,0.25397,0.49265,-0.00256,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,COMMUNITY
1,1,-0.398190,0.92849,1.11940,-0.132170,-0.29808,-0.52848,1.06140,-0.70803,-0.48539,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ENVIRONMENT
2,2,-0.942140,-0.24345,0.20744,0.214930,-1.06640,1.19490,1.52340,-0.16527,0.35522,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RELAX
3,3,-1.074400,-0.88249,0.27640,0.647550,-0.35475,0.25591,-0.70326,-1.00450,1.19360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TRAVEL
4,4,-0.177000,0.43200,0.21391,-0.295150,0.47344,-0.38013,1.60180,0.56238,0.40478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ASTRONOMY


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [5]:
X = df.drop(['tag', 'Unnamed: 0'], axis=1)
y = df['tag']

y = pd.get_dummies(y)

y = y.astype('float32')

y.shape

(25763, 31)

In [6]:
x_shape = X.shape[1]

x_shape

1100

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)

X_train.shape

(20610, 1100)

In [9]:
X_train_reshaped = X_train_np.reshape((-1, 44, 25)) # 1100/25 = 44
X_test_reshaped = X_test_np.reshape((-1, 44, 25))

print(X_train_reshaped.shape)
print(X_test_reshaped.shape)

(20610, 44, 25)
(5153, 44, 25)


In [10]:
import tensorflow as tf

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=(44, 25)))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(1100, activation='sigmoid'))
model.add(tf.keras.layers.Dense(31, activation='softmax'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'] 
    )

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 40, 128)           16128     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 1100)              141900    
                                                                 
 dense_1 (Dense)             (None, 31)                34131     
                                                                 
Total params: 192159 (750.62 KB)
Trainable params: 192159 (750.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32)

Epoch 1/10
645/645 [==============================] - 2s 2ms/step - loss: 2.3219 - accuracy: 0.3363
Epoch 2/10
645/645 [==============================] - 1s 2ms/step - loss: 1.7875 - accuracy: 0.4795
Epoch 3/10
645/645 [==============================] - 1s 2ms/step - loss: 1.6335 - accuracy: 0.5260
Epoch 4/10
645/645 [==============================] - 1s 2ms/step - loss: 1.5238 - accuracy: 0.5570
Epoch 5/10
645/645 [==============================] - 1s 2ms/step - loss: 1.4309 - accuracy: 0.5847
Epoch 6/10
645/645 [==============================] - 1s 2ms/step - loss: 1.3584 - accuracy: 0.6047
Epoch 7/10
645/645 [==============================] - 1s 2ms/step - loss: 1.2888 - accuracy: 0.6260
Epoch 8/10
645/645 [==============================] - 1s 2ms/step - loss: 1.2250 - accuracy: 0.6397
Epoch 9/10
645/645 [==============================] - 1s 2ms/step - loss: 1.1708 - accuracy: 0.6571
Epoch 10/10
645/645 [==============================] - 1s 2ms/step - loss: 1.1229 - accuracy: 0.6678

In [12]:
# Ocena modelu na danych testowych
loss, accuracy = model.evaluate(X_test_reshaped, y_test)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


162/162 [==============================] - 0s 890us/step - loss: 1.7108 - accuracy: 0.5331
Test Loss: 1.7108289003372192
Test Accuracy: 0.5330875515937805
